In [1]:
#It is required to have more than 12GB RAM to perform training
#Subscription for Google Colab Pro might be needed for Google Colab user
#Testing can still be done without training as a pre-trained data is included.
%%capture
#Get current directory and download the training files
import os
curr_dir=os.getcwd()
!wget -O anomaly_detection.zip https://www.dropbox.com/s/bqktvqw4uop8rss/anomaly_detection.zip?dl=0
!unzip anomaly_detection.zip
!rm -rf anomaly_detection.zip
%cd {curr_dir}/anomaly_detection

In [ ]:
#Install requirement. Skip this step if you are running on Google Colab as all the requirement is already installed
#!pip install -r requirements.txt

In [ ]:
##Download the dataset to perform training or testing
#Train dataset
!mkdir -p dev_data 
!wget -O dev_data_ToyCar.zip https://zenodo.org/record/3678171/files/dev_data_ToyCar.zip?download=1
!unzip dev_data_ToyCar.zip -d dev_data
!rm -rf dev_data_ToyCar.zip

#Eval Dataset
!wget -O dev_data_ToyCar.zip https://zenodo.org/record/3727685/files/eval_data_train_ToyCar.zip?download=1
!unzip dev_data_ToyCar.zip -d dev_data
!rm -rf dev_data_ToyCar.zip

In [ ]:
#Start the training
!python 00_train.py --dev 

In [ ]:
#Test the model
!python 01_test.py --dev

In [ ]:
#Convert the model to TFLite with Quantization
!python 02_convert.py --dev

In [ ]:
#Test the converted model
!python 03_tflite_test.py --dev

In [3]:
#Loading the TFLite model
import numpy
import tensorflow as tf
import common as com
mod=com.get_quant_model_name()
tflite_file=f'trained_models/{mod}_model_ToyCar_quant_fullint_micro_intio.tflite'
interpreter = tf.lite.Interpreter(model_path=tflite_file)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
#Selective testing. 2 Slices from normal_id_01_00000200 is selected for inferencing.
#This data will be used to perform inference in hardware

data = com.file_to_vector_array('./dev_data/ToyCar/test/normal_id_01_00000200.wav',n_mels=128,frames=5,
                                n_fft=1024,
                                hop_length=512,
                                power=2.0)
input_scale, input_zero_point = input_details[0]["quantization"]
output_scale,output_zero_point = output_details[0]['quantization']
data_inp = data / input_scale + input_zero_point
input_data = numpy.array(data_inp, dtype=numpy.int8)

#Slice 1 from normal_id_01_00000200.wav
interpreter.set_tensor(input_details[0]['index'], input_data[0:1,:])
interpreter.invoke()
output_data = numpy.empty_like(data_inp)
output_data[0:1,:] = interpreter.get_tensor(output_details[0]['index'])
output_data = (output_data[0:1,:].astype(numpy.float32) - output_zero_point)*output_scale
data_inp_float=(input_data[0].astype(numpy.float32) - input_zero_point)*input_scale
errors = numpy.mean(numpy.square(data_inp_float - output_data[0]))                  
res = numpy.mean(errors)
print(f'Normal Slice 1 Result : {res}')

#Slice 2 from normal_id_01_00000200.wav
interpreter.set_tensor(input_details[0]['index'], input_data[1:2,:])
interpreter.invoke()
output_data = numpy.empty_like(data_inp)
output_data[1:2,:] = interpreter.get_tensor(output_details[0]['index'])
output_data = (output_data[1:2,:].astype(numpy.float32) - output_zero_point)*output_scale
data_inp_float=(input_data[1].astype(numpy.float32) - input_zero_point)*input_scale
errors = numpy.mean(numpy.square(data_inp_float - output_data[0]))                  
res = numpy.mean(errors)
print(f'Normal Slice 2 Result : {res}')

In [ ]:
#Selective testing. 2 Slices from anomaly_id_01_00000250 is selected for inferencing.
#This data will be used to perform inference in hardware

data = com.file_to_vector_array('./dev_data/ToyCar/test/anomaly_id_01_00000250.wav',n_mels=128,frames=5,
                                n_fft=1024,
                                hop_length=512,
                                power=2.0)
input_scale, input_zero_point = input_details[0]["quantization"]
output_scale,output_zero_point = output_details[0]['quantization']
data_inp = data / input_scale + input_zero_point
input_data = numpy.array(data_inp, dtype=numpy.int8)

#Slice 1 from anomaly_id_01_00000250.wav
interpreter.set_tensor(input_details[0]['index'], input_data[0:1,:])
interpreter.invoke()
output_data = numpy.empty_like(data_inp)
output_data[0:1,:] = interpreter.get_tensor(output_details[0]['index'])
output_data = (output_data[0:1,:].astype(numpy.float32) - output_zero_point)*output_scale
data_inp_float=(input_data[0].astype(numpy.float32) - input_zero_point)*input_scale
errors = numpy.mean(numpy.square(data_inp_float - output_data[0]))                  
res = numpy.mean(errors)
print(f'Anomaly Slice 1 Result : {res}')

#Slice 2 from anomaly_id_01_00000250.wav
interpreter.set_tensor(input_details[0]['index'], input_data[1:2,:])
interpreter.invoke()
output_data = numpy.empty_like(data_inp)
output_data[1:2,:] = interpreter.get_tensor(output_details[0]['index'])
output_data = (output_data[1:2,:].astype(numpy.float32) - output_zero_point)*output_scale
data_inp_float=(input_data[1].astype(numpy.float32) - input_zero_point)*input_scale
errors = numpy.mean(numpy.square(data_inp_float - output_data[0]))                  
res = numpy.mean(errors)
print(f'Anomaly Slice 2 Result : {res}')

In [12]:
#Copy and rename the TFLite file to be used by TinyML Generator
import common as com
mod=com.get_quant_model_name()
!cp -R trained_models/{mod}_model_ToyCar_quant_fullint_micro_intio.tflite deep_autoencoder_anomaly_detection.tflite